# Assignment 4 - Kritik Mirg

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re

### Step 1 [Modifying the URL]

In [2]:
job_titles = ['Data Analyst', 'Data Scientist', 'Database Administrator',
             'Machine Learning Engineer', 'Data Engineer']
states = ['Virginia', 'New York', 'California', 
          'Texas', 'Washington State']

In [3]:
links = []
for job in job_titles:
    job_split = job.split()
    for st in states:
        state_split = st.split()
        
        if len(state_split) > 1:
            url = 'https://www.indeed.com/jobs?q=' + job_split[0] + '+' + job_split[1] + '&l=' + state_split[0] + '+' + state_split[1] + '&sort=date'
            
        else: 
            url = 'https://www.indeed.com/jobs?q=' + job_split[0] + '+' + job_split[1] + '&l=' + state_split[0] + '&sort=date'
        
        links.append(url)       
        

In [4]:
for link in links:
    print(link)

https://www.indeed.com/jobs?q=Data+Analyst&l=Virginia&sort=date
https://www.indeed.com/jobs?q=Data+Analyst&l=New+York&sort=date
https://www.indeed.com/jobs?q=Data+Analyst&l=California&sort=date
https://www.indeed.com/jobs?q=Data+Analyst&l=Texas&sort=date
https://www.indeed.com/jobs?q=Data+Analyst&l=Washington+State&sort=date
https://www.indeed.com/jobs?q=Data+Scientist&l=Virginia&sort=date
https://www.indeed.com/jobs?q=Data+Scientist&l=New+York&sort=date
https://www.indeed.com/jobs?q=Data+Scientist&l=California&sort=date
https://www.indeed.com/jobs?q=Data+Scientist&l=Texas&sort=date
https://www.indeed.com/jobs?q=Data+Scientist&l=Washington+State&sort=date
https://www.indeed.com/jobs?q=Database+Administrator&l=Virginia&sort=date
https://www.indeed.com/jobs?q=Database+Administrator&l=New+York&sort=date
https://www.indeed.com/jobs?q=Database+Administrator&l=California&sort=date
https://www.indeed.com/jobs?q=Database+Administrator&l=Texas&sort=date
https://www.indeed.com/jobs?q=Database+Ad

### Step 2: [Extracting the elements]

In [5]:
title = []
company = []
location = []
salary = []
description = []
date = []
rating = []

In [6]:
for url in links:
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    job_container = soup.find_all('div', {'class': {'job_seen_beacon'}})

    for container in job_container:

        try:
            container.find('h2',{'class': 'jobTitle'})
            title_container = container.find('h2',{'class': 'jobTitle'})
            title.append(title_container.text)
        except:
            title.append('Missing')

        try: 
            container.find('span',{'class': 'companyName'}) 
            company_container = container.find('span',{'class': 'companyName'})
            company.append(company_container.text)
        except:
            company.append('Missing')

        try:
            container.find('div',{'class': 'companyLocation'})
            location_container = container.find('div',{'class': 'companyLocation'})
            location.append(location_container.text)
        except:
            location.append('Missing')

        try:
            container.find('div',{'class': 'metadata salary-snippet-container'})
            salary_container = container.find('div',{'class': 'metadata salary-snippet-container'})
            salary.append(salary_container.text)
        except:
            salary.append('Missing')
    
        try:
            container.find('div', {'class':'job-snippet'})
            description_container = container.find('div', {'class':'job-snippet'})
            description.append(description_container.text)
        except:
            description.append('Missing')

        try: 
            container.find('span', {'class': 'date'})
            date_container = container.find('span', {'class': 'date'})
            date.append(date_container.text)
        except:
            date.append('Missing')

        try: 
            container.find('span', {'class': 'ratingsDisplay withRatingLink'})
            rating_container = container.find('span', {'class': 'ratingsDisplay withRatingLink'})
            rating_container_flt = float(rating_container.text)
            rating.append(rating_container_flt)
        except:
            rating.append('Missing')
    

In [7]:
df = pd.DataFrame({'Job title':title,
                          'Company Name': company,
                          'Job location': location,
                          'Salary': salary,
                          'Job description': description,
                          'Date of posting': date,
                          'Rating of the company': rating
                         })
df.head()

,Job title,Company Name,Job location,Salary,Job description,Date of posting,Rating of the company
0,newBusiness Analyst III,"Estes Express Lines, Inc.","Richmond, VA 23230 (Sauer's Gardens area)",Missing,\nMinimum 5 years’ experience designing and de...,PostedToday,3.3
1,newJunior Business Analyst & Tester,Armedia LLC,"Vienna, VA 22182",Missing,"\nIn this position, you may do BA activities, ...",PostedToday,4.4
2,"newDirector, Federal Healthcare Data Analytics",KPMG,"McLean, VA 22102+1 location",Missing,\nUnderstand and articulate technology options...,PostedToday,3.9
3,"newAssociate, Data Scientist - Financial Servi...",KPMG,"McLean, VA 22102",Missing,\nAnalyze and model structured data and implem...,PostedToday,3.9
4,newGlobal Chief Data Office EMS - Metrics and ...,PRICE WATERHOUSE COOPERS,"Richmond, VA 23219 (Central Office area)+1 loc...",Missing,\nLeading engagements with data domain leaders...,PostedToday,4


### Step 3: [Data manipulation]

Editing text in columns by removing all newlines ('\n)

Removing the substring 'new' in the beginning off all Job Titles and 'Posted' from the beginning of Date of posting 

In [8]:
df['Job title'] = df['Job title'].str.replace('new', '', 1)
df['Date of posting'] = df['Date of posting'].str.replace('Posted', '', 1)
df=df.replace('\n','', regex=True)
df.head()

,Job title,Company Name,Job location,Salary,Job description,Date of posting,Rating of the company
0,Business Analyst III,"Estes Express Lines, Inc.","Richmond, VA 23230 (Sauer's Gardens area)",Missing,Minimum 5 years’ experience designing and depl...,Today,3.3
1,Junior Business Analyst & Tester,Armedia LLC,"Vienna, VA 22182",Missing,"In this position, you may do BA activities, bu...",Today,4.4
2,"Director, Federal Healthcare Data Analytics",KPMG,"McLean, VA 22102+1 location",Missing,Understand and articulate technology options a...,Today,3.9
3,"Associate, Data Scientist - Financial Services...",KPMG,"McLean, VA 22102",Missing,Analyze and model structured data and implemen...,Today,3.9
4,Global Chief Data Office EMS - Metrics and Gov...,PRICE WATERHOUSE COOPERS,"Richmond, VA 23219 (Central Office area)+1 loc...",Missing,Leading engagements with data domain leaders t...,Today,4


Splitting Job Location into two seperate columns: 'City' and 'State Information'

In [9]:
df[['City', 'State Information']] = df['Job location'].str.split(',', expand=True)
df.head()

,Job title,Company Name,Job location,Salary,Job description,Date of posting,Rating of the company,City,State Information
0,Business Analyst III,"Estes Express Lines, Inc.","Richmond, VA 23230 (Sauer's Gardens area)",Missing,Minimum 5 years’ experience designing and depl...,Today,3.3,Richmond,VA 23230 (Sauer's Gardens area)
1,Junior Business Analyst & Tester,Armedia LLC,"Vienna, VA 22182",Missing,"In this position, you may do BA activities, bu...",Today,4.4,Vienna,VA 22182
2,"Director, Federal Healthcare Data Analytics",KPMG,"McLean, VA 22102+1 location",Missing,Understand and articulate technology options a...,Today,3.9,McLean,VA 22102+1 location
3,"Associate, Data Scientist - Financial Services...",KPMG,"McLean, VA 22102",Missing,Analyze and model structured data and implemen...,Today,3.9,McLean,VA 22102
4,Global Chief Data Office EMS - Metrics and Gov...,PRICE WATERHOUSE COOPERS,"Richmond, VA 23219 (Central Office area)+1 loc...",Missing,Leading engagements with data domain leaders t...,Today,4,Richmond,VA 23219 (Central Office area)+1 location


Printing percentages of jobs in each city

In [11]:
df['City'].value_counts(normalize=True)*100

New York         15.200000
Seattle          12.533333
Austin            7.733333
San Francisco     5.600000
Dallas            4.533333
                   ...    
Allen             0.266667
Potomac Falls     0.266667
Renton            0.266667
Sacramento        0.266667
Tysons            0.266667
Name: City, Length: 105, dtype: float64

Exporting Dataframe to CSV

In [12]:
df.to_csv('/Users/kritik/Desktop/Mirg_Kritik_INST447_PA4.csv')